In [5]:
#####-------------------------------------------------------------#####
##### FUNCTION TO CREATE A NEW BUCKET OF DATA IN MINIO 
#####-------------------------------------------------------------#####
import minio
from minio.error import S3Error
import json

def create_bucket(bucket_name, minio_credentials):
    """
    Create a bucket in MinIO.

    Parameters:
    - bucket_name (str): The name of the bucket to create.
    - minio_client (minio.Minio): An instance of the Minio client.

    Returns:
    - bool: True if the bucket was created successfully, False otherwise.
    """
    with open(minio_credentials, 'r') as file:
        keys = json.load(file)
        
    minio_client = minio.Minio(
        endpoint = "localhost:9000",
        access_key = keys["accessKey"],
        secret_key = keys["secretKey"],
        secure = True # FIX ME PLEASE!!! WHAT IS THE BEST WAY TO HANDLE THIS SECURITY ISSUE?  
    )
    try:
        # Check if the bucket already exists
        exists = minio_client.bucket_exists(bucket_name)
        if not exists:
            # Make a new bucket
            minio_client.make_bucket(bucket_name)
            print(f"Bucket '{bucket_name}' created successfully.")
        else:
            print(f"Bucket '{bucket_name}' already exists.")
        return True
    except S3Error as e:
        print(f"Error creating bucket: {e}")
        return False


# Create a new bucket
minio_credentials = "credentials_python_minio.json"
create_bucket("dev_rds", minio_credentials)

MaxRetryError: HTTPSConnectionPool(host='localhost', port=9000): Max retries exceeded with url: /dev_rds?location= (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1133)')))